In [1]:
import json
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from DataProcessing.utils import load_yaml
from DataProcessing.extract_graphstate import (
    extract_documents_for_multivectorstore,
    extract_documents_for_multidocstore,
)

config = load_yaml("../config/embedding.yaml")
category_id = config["settings"]["category_id"]
filetype = config["settings"]["filetype"]
edit_path = config["settings"]["edit_path"]
output_path = config["settings"]["output_path"]
os.makedirs(output_path, exist_ok=True)


def load_json_files(path):
    data_list = []
    for filename in os.listdir(path):
        if filename.endswith(".json"):
            with open(os.path.join(path, filename), "r", encoding="utf-8") as file:
                data = json.load(file)
                data_list.append(data)
    return data_list


data_list = []
for category in config["settings"]["category_id"]:
    category_path = os.path.join(edit_path, category, "json")
    data_list.extend(load_json_files(category_path))


vectorstore_documents = []
docstore_documents = []
for data in data_list:
    docstore_document = extract_documents_for_multidocstore(data)
    docstore_documents.extend(docstore_document)
    vectorstore_document = extract_documents_for_multivectorstore(data)
    vectorstore_documents.extend(vectorstore_document)

In [2]:
import uuid
from datetime import datetime
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain_upstage.embeddings import UpstageEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_community.vectorstores.utils import filter_complex_metadata

load_dotenv()
time = datetime.now().strftime("%Y.%m.%d")
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

proj_name = f"2024.09.14_for_multivector_store"
store = LocalFileStore(
    f"/Users/youngseoklee/Desktop/workplace/MacroAgent-withRAG/cache/{proj_name}/data"
)

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=passage_embeddings,
    document_embedding_cache=store,
    namespace=passage_embeddings.model,
)
DB_PATH = f"/Users/youngseoklee/Desktop/workplace/MacroAgent-withRAG/db/2024.09.14_for_multivector_store"
db = Chroma(persist_directory=DB_PATH, embedding_function=cached_embedder)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
id_key = "id_key"
doc_ids = [str(uuid.uuid4()) for _ in docstore_documents]


multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=llm,
)

multi_vector_retriever = MultiVectorRetriever(
    vectorstore=db,
    byte_store=store,
    id_key=id_key,
)

ensemble_retriever = EnsembleRetriever(
    retrievers=[multi_query_retriever, multi_vector_retriever], weights=[0.5, 0.5]
)


summary_docs = [
    Document(page_content=s.page_content, metadata={id_key: doc_ids[i], **s.metadata})
    for i, s in enumerate(vectorstore_documents)
]
filtered_summary_docs = filter_complex_metadata(documents=summary_docs)
filtered_docstore_documents = filter_complex_metadata(documents=docstore_documents)
multi_vector_retriever.vectorstore.add_documents(
    summary_docs, ids=[doc.metadata[id_key] for doc in summary_docs]
)
multi_vector_retriever.docstore.mset(list(zip(doc_ids, docstore_documents)))

In [3]:
retrieved_docs = multi_vector_retriever.invoke("옐런의 계획이 어떻게 되니?")
retrieved_docs

[Document(metadata={'id': '1', 'page': 1, 'images': 'https://post-image.valley.town/Zsa9b23Fl7odR6Ni3Kz_W.png', 'doc_id': '[2024.07.23] 월가아재 시즌3 - 시황칼럼 11편: 잊혀진 옐런이 가지고 있는 한방.md_1', 'datetime': '2024.07.23', 'Header 1': '[2024.07.23] 월가아재 시즌3 - 시황칼럼 11편: 잊혀진 옐런이 가지고 있는 한방', 'Header 2': '1. 자산군별 동향', 'Header 3': '**옐런의 한방**'}, page_content="### **옐런의 한방**  \n트럼프의 토론 우위와 총격 사태에 따라 최근 1~2주 동안 트럼프쪽으로 무게추가 확 기울었고, 시장 움직임도 트럼프가 재선되었을 때를 상정한 섹터 로테이션이나 자산군 가격 변화가 주도했는데요. 그래도 잊어선 안되는 것은 여전히 바이든의 임기는 남아 있고, 옐런의 총알도 남아 있다는 것입니다. 그리고 실질적으로 연말까지 시장 유동성을 좌지우지할 막강한 권한은 트럼프가 아닌 옐런과 파월에게 있는 것이죠. 물론 옐런이 금융시장을 홀로 마음대로 좌지우지할 수 있는 것도 아니고, 바이든의 사퇴처럼 옐런도 예상치 못한 일이 일어나기는 하지만, 그래도 지금부터 대선까지는 파월만큼이나 강력한 플레이어입니다. 오히려 파월은 중립이어야할 의무에 명시적으로 묶여 있는데다 보는 눈이 많은 만큼, 옐런의 운신의 폭과 그에 따른 영향력은 파월보다 적어도 앞으로 6개월 동안만큼은 더 클 수도 있는 것이죠.  \n파월 쪽의 경우, 저는 감으로는 파월이 트럼프를 싫어할 거라 생각하지만 어쨌든 표면적으로 드러나지 않은 사실을 기반해서 논리를 전개하는 것은 투자 의사판단에서 매우 위험한 일입니다. 그렇기 때문에 파월 쪽은 Fedwatch에서 드러난 컨센서스, 9월 1회 금리 인하를 디폴트로 두고 생각하는 것이 합리적이라고 생각하고 있습니다. 올해 초까지만 해도 

In [14]:
from langchain.callbacks.manager import CallbackManagerForRetrieverRun


def combined_retriever(query):
    # CallbackManagerForRetrieverRun 인스턴스 생성
    run_manager = CallbackManagerForRetrieverRun.get_noop_manager()

    multi_queries = multi_query_retriever.generate_queries(
        question=query, run_manager=run_manager
    )
    all_docs = []
    for q in multi_queries:
        docs = multi_vector_retriever.invoke(q)
        all_docs.extend(docs)

    # 중복 제거를 위해 딕셔너리를 사용
    unique_docs = {
        (doc.page_content, frozenset(doc.metadata.items())): doc for doc in all_docs
    }.values()

    return sorted(
        unique_docs, key=lambda x: x.metadata.get("relevance_score", 0), reverse=True
    )


query = "옐런의 계획이 어떻게 되니?"
combined_results = combined_retriever(query)
ensemble_results = ensemble_retriever.get_relevant_documents(query)

In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import pandas as pd
import sys
import os
import json
from langchain_anthropic import ChatAnthropic

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from DataProcessing.utils import load_yaml


path = "./data/custom_testdataset.xlsx"
# testset = pd.read_csv(path)
testset = pd.read_excel(path)

questions = testset["question"].to_list()
ground_truth = testset["ground_truth"].to_list()

data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

prompt_template = load_yaml("../prompts/Retriever._prompt.yaml")["prompt"]
prompt = PromptTemplate.from_template(prompt_template)
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.5)
rag_chain = (
    {"context": multi_vector_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


for query in questions:
    data["question"].append(query)
    data["answer"].append(rag_chain.invoke(query))
    data["contexts"].append(
        [doc.page_content for doc in multi_vector_retriever.invoke(query)]
    )

path = "./data/customtestset_sonnet_multivector.json"
with open(path, "w") as file:
    json.dump(data, file)